# Notebook to run the gaussian regularized alghorithms 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from prob_models.utils.utils import read_add_paths, read_paths, getdata

import prob_models.cov_estim.estimators as es 
import prob_models.cov_estim.run_parallel as run
import prob_models.cov_estim.utils as utils
import os
import importlib
from prob_models.cov_estim.utils import *

importlib.reload(utils)
importlib.reload(run)

<module 'prob_models.cov_estim.run_parallel' from '/Users/ariannaarmanetti/Smastruzzing/BrainHack2024/prob-models/prob_models/cov_estim/run_parallel.py'>

In [3]:
nsamples = 200
nfeatures = 116

X = np.random.randn(nsamples, nfeatures)  # dummy dataset with true covariance I
Xtrain = X[:int(0.8*nsamples)]
Xtest = X[int(0.8*nsamples):]

res = es.fit_PCA(Xtrain, Xtest, ncomponents=10)
# res = es.fit_PCA_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_PCA_Minka(Xtrain, Xtest)
# res = es.fit_ConservativePCA(Xtrain, Xtest, p=10)
# res = es.fit_ConservativePCA_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_FactorAnalysis(Xtrain, Xtest)
# res = es.fit_FactorAnalysis_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_GraphicalLasso(Xtrain, Xtest, alpha=0.1)
# res = es.fit_GraphicalLasso_CV(Xtrain, Xtest, n_jobs=None)
# res = es.fit_Shrinkage(Xtrain, Xtest, shrinkage=0.1)
# res = es.fit_Shrinkage_CV(Xtrain, Xtest, shrinkages=None,  cv_scoring="likelihood", n_jobs=None)
# res = es.fit_RIE(Xtrain, Xtest)
# res = es.fit_RIE_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_GradientAscent(Xtrain, Xtest, bootstrapping=False, stop='completion', val_frac=0.1)
# res = es.fit_GradientAscentWishart(Xtrain, Xtest, bootstrapping=False, stop='completion', val_frac=0.1)

C = X.T @ X / nsamples # empirical covariance
Cclean = res["Cclean"] # cleaned covariance


In [4]:
#load data
paths=read_paths('/Users/ariannaarmanetti/Smastruzzing/Brainhack2024/prob-models/paths.txt')
datadic=getdata(paths[-1])
print(paths[-1])

T,N=np.shape(datadic['032302'])
print(T,N)





/Users/ariannaarmanetti/Downloads/data/iPA_183/ts/
652 183


In [5]:
np.shape([el for el in datadic.values()]) 

(136, 652, 183)

In [8]:
resfile = 'all_results_BIO.pickle'

train_fraction = 0.8
results={}

for i,( key, X) in enumerate(datadic.items()):
    if i<3:
        print (i)
        # Compute the corr mat C obtained flattening the training data from all patients
        Cbar = computeCbar([el for el in datadic.values()], train_fraction=train_fraction)
        Xtrain, Xtest = split_train_test(X, train_fraction=train_fraction, standardize=True, seed=i)
        key,value=run.single_run(i, Xtrain, Xtest, Cbar=Cbar)
        results[key]=value

0
1
2


In [9]:
for key, val in results[0].items():
    print (key,val['test_likelihood'])

Shrink_CV_likelihood 161.4996712066818
RIE_CV_likelihood 143.03761108463695
Shrink_CV_completion_error 166.39817622180493
RIE_CV_completion_error 171.55130595277646
